## Building the Source Table(s)

This notebook describes how the `Source` tables are constructed, starting with the **Stripe 82** dataset and followed the **XMM-LSS** dataset. The two `Source` tables are merged into one in the end. Much of the information listed here should be a replicate of the [01_Building_the_Object_Table.ipynb](01_Building_the_Object_Table.ipynb) notebook, thus we will reference that notebook wherever appropriate. 

__Note:__ This table is different than the 'ForcedSource' table, which is the place to get the light curves. Here, we provide at most one photometry (in each band) for every object, except in the XMM-LSS region where both DES and SDSS photometries are available.

### **1. Stripe 82**
```bash
-> s82SourceTable.parquet
```

### 1.1 IDs
- `sourceId`: Unique source identifier
- `objectId`: ID of the Object this source was associated with, if any.

### 1.2 Astrometry
- `ra`, `dec`: The position of the matched photometry. For Stripe 82, they are SDSS DR16 positions. 

### 1.3 Photometry
- `psMag`, `psMagErr`
- `bdMag`, `bdMagErr`

We matched our sources to the SDSS DR16 photoprimary catalog. The `psMag` column is populated using the 'psfmag' and the 'bdMag' is populated using the 'modelmag'.

- `psFlux`, `psFluxErr`
- `bdFlux`, `bdFluxErr`

The fluxes are converted from asinh magnitudes following the procedure described in Section 1.2 of the [01_Building_the_Object_Table.ipynb](01_Building_the_Object_Table.ipynb) notebook.

### 1.4 Extendedness (morphology)
- `extendedness`: See the [01_Building_the_Object_Table.ipynb](01_Building_the_Object_Table.ipynb) notebook.

### 1.5 flags
Only SDSS photometries are used for the Stripe 82 source table, thus, we set the `flags` column to 0 if a match is found for a particular object, otherwise NaN.

__Note:__ 
Since a series of cuts were already applied while obtaining the SDSS photometry (see [here](https://www.sdss.org/dr16/tutorials/flags/) for the exact cuts used), thus we presume that SDSS photometries are already 'clean'. 

### 1.6 filter
This column won't be provided in the nominal 'Source' table either, but it can be obtained by joining with other tables.

### __2. XMM-LSS__
```bash
-> xmmlssSourceTable.parquet
```

### 2.1 IDs
- `sourceId`: Unique source identifier
- `objectId`: ID of the Object this source was associated with, if any.

### 2.2 Astrometry
- `ra`, `dec`: The position of the matched photometry. 

For u band photometry, they are SDSS DR16 positions; for photometry in 'grizy' band, they are DES DR2 positions.

### 2.3 Photometry
- `psMag`, `psMagErr`
- `bdMag`, `bdMagErr`

We matched our sources to the SDSS DR16 photoprimary catalog and the DES DR2 catalog. 

1. The point-source magnitude (`psMag`) is filled by 'psfmag' from SDSS in the u band and 'wavg_mag_psf' (weighted-average single epoch PSF magnitude) from DES in the 'grizy' bands. 

2. The extended-source magnitude (`bdMag`) is filled using 'modelmag' from SDSS in the u band and 'magauto' (measured on coadded images) from DES in the 'grizy' bands.
    
- `psFlux`, `psFluxErr`
- `bdFlux`, `bdFluxErr`

The fluxes are converted from asinh magnitudes (or AB magnitudes) following the procedure described in Section 1.2 of the [01_Building_the_Object_Table.ipynb](01_Building_the_Object_Table.ipynb) notebook.

### 2.4 Extendedness (morphology)
- `extendedness`: See the [01_Building_the_Object_Table.ipynb](01_Building_the_Object_Table.ipynb) notebook.

### 2.5 flags
- SDSS ('ugriz' bands): 0 if match is found and used.
- DES ('grizy' bands): 0 if DES photometry flag <= 3 (as recommended)

__Note:__ Since a series of cuts were already applied while obtaining the SDSS photometry (see [here](https://www.sdss.org/dr16/tutorials/flags/) for the exact cuts used), thus we presume that SDSS photometries are already 'clean'. 

### 2.6 filter
This column won't be provided in the nominal 'Source' table either, but it can be obtained by joining with other tables.

### 3. The 'Source' Table
```bash
-> SourceTable.parquet
```
The last step is to merge the __Stripe 82__ source table and the __XMM-LSS__ source table. We did that by matching the __XMM-LSS__ objects with the __Stripe 82__ objects using a 0.5 arcsec radius. The Stripe 82 sources that have counterparts in the XMM-LSS sample are re-assigned the objectId using that from their counterparts in the XMM-LSS sample. 